In [7]:
import os
os.environ["SERPAPI_API_KEY"]="c883f29263611e9852a938faf22bdbb015f41473b57563d4842950556bafebba"

In [6]:
!python -m pip install python-dotenv pathvalidate markdownify puremagic mammoth python-pptx easyocr pdfminer.six pypdf easyocr google-search-results -qqq
# !python -m pip install torch==2.3.0 -qqq

In [12]:
from transformers.agents import ReactJsonAgent, HfEngine
from utils.web_surfer import *
from transformers.agents.prompts import DEFAULT_REACT_JSON_SYSTEM_PROMPT
import shutil

WEB_TOOLS = [
    SearchInformationTool(),
    NavigationalSearchTool(),
    VisitTool(),
    DownloadTool(),
    PageUpTool(),
    PageDownTool(),
    FinderTool(),
    FindNextTool(),
]

websurfer_llm_engine=HfEngine("http://127.0.0.1:8087")

surfer_agent = ReactJsonAgent(
    llm_engine=websurfer_llm_engine,
    tools=WEB_TOOLS,
    max_iterations=12,
    verbose=1,
    system_prompt=DEFAULT_REACT_JSON_SYSTEM_PROMPT + "\nAdditionally, if after some searching you find out that you need more information to answer the question, you can use `final_answer` with your request for clarification as argument to request for more information.",
)


In [13]:
class SearchTool(Tool):
    name = "ask_search_agent"
    description = "A search agent that will browse the internet to answer your query. Ask him for all your web-search related questions, but he's unable to do problem-solving. Provide him as much context as possible, in particular if you need to search on a specific timeframe!"

    inputs = {
        "query": {
            "description": "Your query, as a natural language sentence. You are talking to an human, so provide them with as much context as possible! ",
            "type": "text",
        }
    }
    output_type = "text"
    already_started = False

    def forward(self, query: str) -> str:
        return surfer_agent.run(query + (f"\nYour browser is already open to the page '{browser.page_title}' at the address '{browser.address}'." if self.already_started else ""))


class ZipInspectorTool(Tool):
    name = "extract_inspect_zip_folder"
    description = "Use this to extract and inspect the contents of a zip folder."
    inputs = {
        "folder": {
            "description": "The path to the zip folder you want to inspect.",
            "type": "text",
        }
    }
    output_type = "text"

    def forward(self, folder: str) -> str:
        folder_name = folder.replace(".zip", "")
        os.makedirs(folder_name, exist_ok=True)
        shutil.unpack_archive(folder, folder_name)

        # Convert the extracted files
        result = "We extracted all files from the zip into a directory: find the extracted files at the following paths:\n"
        for root, dirs, files in os.walk(folder_name):
            for file in files:
                result += f"- {os.path.join(root, file)}\n"

        return result